# Import

In [1]:
variable_folder = "stored_variables/"

In [2]:
#Import builtin
import os
import sys
import time
from datetime import datetime, date, time
import copy
import json

#sci and num py
import numpy as np
import scipy.optimize as optimize
from scipy.optimize import least_squares
from scipy.interpolate import Rbf
from scipy import interpolate
import scipy
import scipy.io as sio
from numpy.linalg import norm
from numpy.linalg import inv

import pandas as pd

#okada wrapper is crucial
try: from okada_wrapper import dc3d0wrapper, dc3dwrapper 
except: print('Please install the Okada Wrapper module.\
            https://github.com/tbenthompson/okada_wrapper') 

# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.patches as patches
from matplotlib.ticker import AutoMinorLocator
import matplotlib.mlab as mlab
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
import matplotlib.path as mpltPath
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

# Import plotly related stuff
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from scipy.spatial import Delaunay
import matplotlib.cm as cm
from functools import reduce

# For clustering
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import sklearn.mixture
import multiprocessing #!# may not need multiprocessing
import sklearn

# pyproj handles UTM projection
import pyproj

In [3]:
# Get clustering info
%run ./clustering_base_file.ipynb

In [4]:
# Functions for loading data. 
%run ./Load_Data.ipynb

In [5]:
# Get functions that are used for many mathematical calculations. Open that file for details. 
%run ./Calculation_Functions.ipynb

In [6]:
# Functions for fault splitting 
%run ./Fault_Splitting.ipynb #!# remove this

In [7]:
# Get some fundamental functions for making figures. Open that file for details. 
%run ./Figure_Making_Functions.ipynb

In [8]:
# Get some stress related stuff
%run ./stress_base_file.ipynb

# Start processing

## Load Hypocenters

In [9]:
# load sequence data 

allHypo = hypoData(selectSequence = '201617')
allHypo.geogToMet()

allHypo.labels=np.zeros(allHypo.mL.size)

if False:
    allHypo.plot3dclusters(pointSize=1, save=False)

FileNotFoundError: File b'focmec-gmt-utm-medi.reloc.dat' does not exist

## How do they distribute through time?

In [ ]:
times = allHypo.time.astype('float')
labs = np.zeros(times.shape)
windows = 10
twinds = np.linspace(np.amin(times), np.amax(times), windows)
for i, t in enumerate(twinds[:-1]):
    labs[times>t] = i

In [ ]:
fig = plt.figure()
plt.axis('off')
plotlabels = [twinds[i] for i in np.arange(windows)]
xsLeg = np.zeros(twinds.size); ysLeg = twinds
plt.scatter(xsLeg, ysLeg, c = twinds, cmap=cm.jet)
for i, txt in enumerate(plotlabels):
    plt.annotate(s = txt, xy = (xsLeg[i], ysLeg[i]) )

In [ ]:
ind = None
keeper = labs != ind


In [ ]:
if True:
    # allHypo.plot3dclustersLeg = plot3dclustersLeg
    allHypo.plot3dclusters(
            x = allHypo.x[keeper], y = allHypo.y[keeper], z = allHypo.z[keeper],
            labels = labs[keeper],
            pointSize=1
        )

In [ ]:
# plot temporal change in time / direction of fault dimension

## Spectral Clustering

In [ ]:
# Spectral clustering
sp = spClust(minMag = 3, selectSequence='201617')
sp.assign_labels="discretize"
sp.n_init=10
sp.beta=1
sp.n_clusters=10 # choose this before hand based on the 3 main faults

sp.cluster()
# spectral clustering is completed 
# sp.labels indicates cluster identities
if False:
    sp.plot3dclusters(pointSize=7)

In [ ]:
if False:
    sp.plot3dclusters(pointSize=2, save = False, lineWidthR=1e-10, 
        x = sp.x, y = sp.y, z = sp.z, labels = sp.labels
                     )

## DBSCAN Clustering

In [ ]:
# DBSCAN clustering
db = dbClust(selectSequence='201617')
db.eps=1 * 1e3
db.min_samples=60
db.fitdbscan() # cluster in modified axis system

# plt.hist(db.labels)
plt.show()
# print(np.sum(db.labels==-1))

In [ ]:
# all points including noise
if False:
    db.plot3dclusters(pointSize=.7, save = False, name = 'dbscannewdata', lineWidthR=1e-10)

In [ ]:
# def plot3dclusters(self, pointSize=4, lineWidthR = 1/10, save = False, name = 'to_delete', 
#                       x=None, y=None, z=None, labels=None):
#         if x is None:
#             x = self.x
#             y = self.y
#             z = self.z
#             labels = self.labels
        
#         scatterPlot = [go.Scatter3d(
#             x=x,
#             y=y,
#             z=z,
#             mode='markers',
#             marker=dict(
#                 size=pointSize,
#                 color=labels,
#                 colorscale='Jet',
#                 line=dict(
#                         color='rgb(0,0,0)',
#                         width=lineWidthR 
#                     ),
#                 opacity=1) )]
        
#         ############################ 
#         textFontDict = {'color':'red', 'size':18}
#         ### north arrow
# #         xNA = 6.5e3; yNA = 10e3; zNA = -14e3
#         xR = 10e3
#         xNA = np.amax(xR-3e3)
#         yNA = 28e3#np.amax(self.y)
#         zNA = -14e3#np.amin(self.z)
#         yBase = yNA - 5e3
#         northLine = go.Scatter3d(
#             x=[xNA, xNA], y=[yBase, yNA], z=[zNA, zNA],
#             text = np.array(['', 'N']),
#             mode='lines+text',#Change?
#             line  =dict(width = 5, color = '#7f7f7f'),
#             textposition = 'middle left',
#             textfont = textFontDict
#         )

#         northCone = dict(
#             type = 'cone',
#             colorscale = 'Greys',
#             showscale = False,
#             x = [xNA], y = [yNA], z = [zNA],
#             u = [0], v = [2e3], w = [0]
#             )
        
#         upLine = go.Scatter3d(
#             x=[xNA, xNA], y=[yBase, yBase], z=[zNA, zNA+5e3],
#             text = np.array(['', 'UP']),
#             mode='lines+text',#Change?
#             line  =dict(width = 5, color = '#7f7f7f'),
#             textposition = 'middle left',
#             textfont = textFontDict
#         )
        
#         upCone = dict(
#             type = 'cone',
#             colorscale = 'Greys',
#             showscale = False,
#             x = [xNA], y = [yBase], z = [zNA+5e3],
#             u = [0], v = [0], w = [2e3]
#             )
#         ###
        
#         ###  scale bar      
#         xN = np.array([xR, xR, xR])
#         yN = np.array([-10e3, -5e3, 0])+yNA
#         zN = np.array([zNA, zNA, zNA])
#         scaleBar = go.Scatter3d(
#             x=xN,
#             y=yN,
#             z=zN,
#             text = np.array(['', '10 km', '']),
#             mode='lines+text',
#             line  =dict(width = 15, color = '#7f7f7f'),
#             textposition='middle right',
#             textfont = textFontDict
#         )
#         ###
#         scatterPlot.append(scaleBar)
#         scatterPlot.append(northLine)
#         scatterPlot.append(northCone)
#         scatterPlot.append(upLine)
#         scatterPlot.append(upCone)
#         #########################
        
#         layout = go.Layout(
#             showlegend=False,
#             autosize=False,
#             width=1000,
#             height=1000,
#             scene=dict(
#                 camera=dict(eye=dict(x=1.75, y=-0.7, z= 0.75) ),
#                 aspectmode = 'data',
#                 xaxis = dict(showticklabels=False, title=''),
#                 yaxis = dict(showticklabels=False, title=''),
#                 zaxis = dict(showticklabels=False, title='')
#                       )  
#             )

#         fig2 = go.Figure(data=scatterPlot, layout=layout)
        
#         if not save:
#             iplot(fig2)
#         elif save:
#             py.iplot(fig2, filename = name)

In [ ]:
inc = db.labels!= -1
# inc = np.ones(inc.shape, dtype = 'bool')
if False:
    db.plot3dclusters(pointSize=1.2, save = False, lineWidthR=1e-10, 
        x = db.x[inc], y = db.y[inc], z = db.z[inc], labels = db.labels[inc]
                     )

In [ ]:
if False:
    db.plot3dclusters(pointSize=1.2, save = False, lineWidthR=1e-10, 
        x = db.x[inc], y = db.y[inc], z = db.z[inc], labels = allHypo.time[inc].astype('float')
                     )

## Agglomerative Clustering

In [ ]:
# Agglomerative clustering

class agClust(hypoData):
    def __init__(self, minMag=None, selectSequence = None):
        super().__init__(minMag=minMag, selectSequence = selectSequence)
        
        self.n_clusters=2
        self.affinity='euclidean'
        self.memory=None
        self.connectivity=None
        self.compute_full_tree='auto'
        self.linkage='ward'
        
        
    def fitAgClust(self):        
        self.pTrans = sklearn.preprocessing.StandardScaler().fit_transform(self.p)
        
        self.output = sklearn.cluster.AgglomerativeClustering(
            self.n_clusters,
            affinity=self.affinity,
            memory=self.memory,
            connectivity=self.connectivity,
            compute_full_tree=self.compute_full_tree,
            linkage=self.linkage
            ).fit(self.p)
        
        self.labels=self.output.labels_
 
minMag = 3
ag = agClust(minMag = minMag, selectSequence = '201617')
ag.n_clusters = 15
ag.fitAgClust()

In [ ]:
if True:
    ag.plot3dclusters(pointSize=1.5)

## Makeing surfaces

Here, I am trying to construct 3D surfaces to fit clusters. It isn't ready yet. 

In [ ]:
# class manySurfaces3D:
#     def __init__(self, terp, allHypo, splineNodesLine = 7):
#         self.setTerp(terp)
#         self.allHypo = allHypo
#         self.splineNodesLine = splineNodesLine # temporary
        
#     def setTerp(self, terp):
#         self.terp = terp
        
#         if terp.minMag is None:
#             terp.minMag = -1e9 #arbitrarily low
#         self.keepMag = allHypo.mL>=terp.minMag
#         self.keepGen = self.keepMag
        
#     def interpolation(self, labelBounds = (0, None)):
#         self.labelBounds = labelBounds
#         self.allLabels = np.unique(self.terp.labels)
#         self.labelsUse = self.allLabels[ 
#             (self.allLabels>=self.labelBounds[0]) * 
#             (self.allLabels< self.labelBounds[1]) ]
#         self.surfObjs = []
        
#         for lab in self.labelsUse:
#             booFault = self.terp.labels==lab

#             xi = self.allHypo.x[self.keepGen][booFault]
#             yi = self.allHypo.y[self.keepGen][booFault]
#             zi = self.allHypo.z[self.keepGen][booFault] 

#             surfacei = interp(xi, yi, zi,
#                            eps=.01, minPlaneDist = 10000,exp=2,
#                            splineNodesStrike = splineNodesLine, # Change
#                               splineNodesDip = splineNodesLine) # Change
            
#             self.surfObjs.append(surfacei)
                
#     def makeSurfaces(self, gridNodesStrike = 40, gridNodesDip = 24):     
#         for lab in self.labelsUse:
#             self.surfObjs[lab].gridNodesStrike = gridNodesStrike
#             self.surfObjs[lab].gridNodesDip = gridNodesDip

#             #the outermost cell center be inset from the outermost part of surface:
#             shiftS = (np.amax(self.surfObjs[lab].pS) - np.amin(self.surfObjs[lab].pS)
#                      ) / self.surfObjs[lab].gridNodesStrike * .5
#             shiftD = (np.amax(self.surfObjs[lab].pD) - np.amin(self.surfObjs[lab].pD)
#                      ) / self.surfObjs[lab].gridNodesDip * .5
            
#             # Generate points on surface
#             self.surfObjs[lab].splineInterp(
#                 cutPoints=True, sEdgeShift = shiftS, dEdgeShift = shiftD)
            
            
#     def plotManySurfaces3D(self, pointSize = 2):
#         data = []
#         for lab in self.labelsUse:
#             shape = (self.surfObjs[lab].gridNodesStrike, 
#                      self.surfObjs[lab].gridNodesDip)
            
#             iS = self.surfObjs[lab].interpS
#             iD = self.surfObjs[lab].interpD
#             xs = self.surfObjs[lab].interpX
#             ys = self.surfObjs[lab].interpY 
#             zs = self.surfObjs[lab].interpZ
#             x = self.surfObjs[lab].x
#             y = self.surfObjs[lab].y
#             z = self.surfObjs[lab].z
            
#             points2D=np.vstack([iS,iD]).T
#             tri=Delaunay(points2D)

#             surfacePlot=plotly_trisurf(xs, ys, zs,
#                                        tri.simplices, colormap=cm.cubehelix, plot_edges=None)


#             scatterPlot = go.Scatter3d(x=x, y=y, z=z,
#                             mode='markers',
#                             marker=dict(
#                                 size=pointSize,
#         #                         color=labels,
#                                 colorscale='Jet',
#                                 line=dict(
#                                         color='rgb(0,0,0)',
#                                         width=1 
#                                     ),
#                                 opacity=1) ) 
#             surfacePlot.append(scatterPlot)

#             data = data + surfacePlot

#         layout = go.Layout( scene=dict( aspectmode = 'data' ) )

#         fig = go.Figure(data=data, layout=layout) 
#         iplot(fig)

# #     plotManySurfaces3D(surfObjs, allLabels)

        
# instance = manySurfaces3D(db, allHypo)   
# instance.interpolation(labelBounds = (0, 5) )
# instance.makeSurfaces()
# instance.plotManySurfaces3D()

# Increase dimensions

## How to quantify similarity in orientation

In [ ]:
def vectSimilarity(v1, v2 = None, under90 = False,
            dotMethod = True):
    # similarity is from 0 to 1 if v1 and v2 are unit vectors
    if v2 is None:
        v2 = v1
            
    sim = np.matmul(v1, v2.T) # dot products
            
    if under90:
        sim = np.abs(sim)
            
    if dotMethod:
        return sim
    
    else:
        sim[sim>1] = 1
        sim[sim<-1] = -1
        angles = np.arccos(sim)
        del sim
        return angles

I can use the angle between focal mechanism planes. but I can't simply add them as they always sum to the same number. Instead, I need to norm the reslults. The PROBLEM is that it turns out that well aligned focal mechanisms give a higher angle mismatch if I try to count all 4 combinations of possible focal correlations. 

In [ ]:
points = 180
print('angles go from 0 to 180')

s1f = np.linspace(0, np.pi, points)
d1f = np.zeros(points) + np.pi / 2
r1f = np.zeros(points)
sdr = [[s1f, d1f, r1f], [s1f+np.pi/2, d1f, r1f]]
def orientSimAngle(sdr, normexp = 2):
    orientSim = [[],[]]
    under90=True
    for i in range(2):
        for j in range(2):
            ni, __ = faultUnitVectors(*sdr[i])
            nj, __ = faultUnitVectors(*sdr[j])
            simij = vectSimilarity(ni, nj, under90=under90, dotMethod = False)
            orientSim[i].append(simij)

    finOrientSim = np.zeros(orientSim[0][0].shape)
    for i in range(2):
        for j in range(2):
            finOrientSim+=orientSim[i][j] ** normexp

    finOrientSim = finOrientSim ** (1/normexp)
    finOrientSim -= np.amin(finOrientSim)
    finOrientSim *= 1/np.amax(finOrientSim)
    return finOrientSim
finOrientSim = orientSimAngle(sdr)
plt.imshow(finOrientSim, interpolation = 'bilinear', origin  = 'lower')
plt.colorbar()
plt.show()

It turns out that the dot product is lowest for well aligned focal mechanisms. Thus, this doesn't seem to work either. 

In [ ]:
def dotToAffin(dot, beta):
    dot[dot>1.] = 1
    angles = np.arccos(dot) # assuming only unit vectors were used
    ret = np.exp(-beta * angles / angles.std())
    return ret

def orientSimAffin(sdr, normexp = 1, beta = 1):
    print('should beta be negative somewhere?')
    orientSim = [[],[]]
    under90=True
    for i in range(2):
        for j in range(2):
            ni, __ = faultUnitVectors(*sdr[i])
            nj, __ = faultUnitVectors(*sdr[j])
            simij = vectSimilarity(ni, nj, under90=under90, dotMethod=True)
#             simij = dotToAffin(simij, beta = beta)
            orientSim[i].append(simij)

    finOrientSim = np.zeros(orientSim[0][0].shape)
    for i in range(2):
        for j in range(2):
            finOrientSim+=orientSim[i][j] ** normexp

    finOrientSim = finOrientSim ** (1/normexp)
    finOrientSim -= np.amin(finOrientSim)
    finOrientSim *= 1/np.amax(finOrientSim)
    return finOrientSim
finOrientSim = orientSimAffin(sdr)
plt.imshow(finOrientSim, interpolation = 'bilinear', origin  = 'lower')
plt.colorbar()
plt.show()

Another option would be to take the dot product of normals and find only one possible orientation match rather than include both

Another option would be to find what is the minimum angle I could rotate one focal mechanism to get the second focal mechanism. The lower the angle, the better the fit. 

Possibly there is already a formulation to determine the similarity of two planes (that is orientation as well as position). 

## Quantify similarity in time

In [ ]:
selectSequence = '201617'
allHypo = hypoData(selectSequence=selectSequence)  
        
FA = ~( (allHypo.st1 == 0) * (allHypo.st2 == 0) )

In [ ]:
beta = 1
t=allHypo.time[FA]
timeDist = np.zeros((t.size, t.size))
for i in np.arange(t.size):
    timeDist[i] = np.abs(t[i] - t)
timeDist -= timeDist.min()
timeDist *= 1/timeDist.max()
timeAffin = np.exp(-beta * timeDist / timeDist.std())

## Similarity matrix in spectral

In [ ]:
# second swarm
beta = 1
n_clusters = 5
n_init = 10
assign_labels="discretize"
gamma=1

selectSequence = '201617'
allHypo = hypoData(selectSequence=selectSequence)  
        
FA = ~( (allHypo.st1 == 0) * (allHypo.st2 == 0) )
s1 = allHypo.st1[FA]
d1 = allHypo.dp1[FA]
r1 = allHypo.rk1[FA]
s2 = allHypo.st2[FA]
d2 = allHypo.dp2[FA]
r2 = allHypo.rk2[FA]
sdr = [[s1, d1, r1], [s2, d2, r2]]

finOrientSim = orientSimAffin(sdr)
        
dists = distances(
          allHypo.x[allHypo.FA],
          allHypo.y[allHypo.FA],
          allHypo.z[allHypo.FA],
          allHypo.x[allHypo.FA],
          allHypo.y[allHypo.FA],
          allHypo.z[allHypo.FA]
         )
distSim = np.exp(-beta * dists / dists.std())

finSim = timeAffin*distSim*finOrientSim

spBase = sklearn.cluster.SpectralClustering(

        n_clusters=n_clusters,
        n_init=n_init,
        assign_labels=assign_labels,
        random_state=np.random.RandomState(1),
        gamma=1,
        affinity='precomputed'

        )

result = spBase.fit(finSim)

sp = spClust(minMag = 0, selectSequence=selectSequence)
sp.results = result
sp.labels = result.labels_

if False:
    sp.plot3dclusters(pointSize=3,
                     x=allHypo.x[allHypo.FA],
                     y=allHypo.y[allHypo.FA],
                     z=allHypo.z[allHypo.FA],
                     labels=sp.labels,
                     
                     save = False, name = 'newData')

## Similarity matrix in DBSCAN

In [ ]:
selectSequence = '201617'
allHypo = hypoData(selectSequence=selectSequence)
class dbClust(hypoData):
    def __init__(self, minMag=None, selectSequence = None):
        super().__init__(minMag=minMag, selectSequence = selectSequence)
        self.sort=False
        
    def fitdbscan(self, weight=None):        
#         self.pTrans = sklearn.preprocessing.StandardScaler().fit_transform(self.p)
        self.pTrans=self.p
        
        if weight is None:       
            self.db = sklearn.cluster.DBSCAN(
                eps=self.eps, min_samples=self.min_samples, 
                n_jobs=multiprocessing.cpu_count()-1
                ).fit(self.pTrans)
        elif weight is not None:
            self.db = sklearn.cluster.DBSCAN(
                eps=self.eps, min_samples=self.min_samples, 
                n_jobs=multiprocessing.cpu_count()-1
                ).fit(self.pTrans, weight)   
        
        self.labels=self.db.labels_
        
    def setLabels(self, labels):
        self.labels = labels

In [ ]:
sdr = [[s1, d1, r1], [s2, d2, r2]]
orientSim = [[],[]]
under90=True
for i in range(2):
    for j in range(2):
        ni, __ = faultUnitVectors(*sdr[i])
        nj, __ = faultUnitVectors(*sdr[j])
        simij = vectSimilarity(ni, nj, under90=under90)
        orientSim[i].append(simij)
        
dists = distances(
          allHypo.x[allHypo.FA],
          allHypo.y[allHypo.FA],
          allHypo.z[allHypo.FA],
          allHypo.x[allHypo.FA],
          allHypo.y[allHypo.FA],
          allHypo.z[allHypo.FA]
         )

def stretchDimensions(p):
    """p shape should be (points, dimensions)"""
    pN = np.zeros(p.shape)
    for i in np.arange(p.shape[1]):
        pN[:,i]  = p[:,i]/(np.amax(p[:,i])-np.amin(p[:,i]) )
        pN[:,i] -= np.amin(pN[:,i])
        
    return pN

p = np.array([
    allHypo.x[allHypo.FA],
    allHypo.y[allHypo.FA],
    allHypo.z[allHypo.FA]
    ]).T

pN = stretchDimensions(p)

distsN = distances(pN[:,0],
                  pN[:,1],
                  pN[:,2],
                  pN[:,0],
                  pN[:,1],
                  pN[:,2])

# finOrientSim = np.zeros(orientSim[0][0].shape)
# for i in range(2):
#     for j in range(2):
#         print('changing orientSim')
#         finOrientSim+=orientSim[i][j] ** 2
# finOrientSim = np.sqrt(finOrientSim)
        
# # finOrientSim *= 1/4

# orientDist = 1-finOrientSim

In [ ]:
finOrientSim = np.zeros(orientSim[0][0].shape)
normexp = 2
for i in range(2):
    for j in range(2):
        print('changing orientSim')
        finOrientSim+=orientSim[i][j] ** normexp
finOrientSim = finOrientSim ** (1/normexp)
plt.hist(finOrientSim[0])

In [ ]:
# not implimented yet, this is the next task
# DBSCAN clustering
dbdim = dbClust(selectSequence='201617')
# dbdim.fitdbscan() # cluster in modified axis system
# plt.hist(dbdim.labels)
eps = .1#* 1e3
min_samples = 15

dbR = sklearn.cluster.DBSCAN(eps = eps, min_samples=min_samples, metric = 'precomputed')

pFinal = distsN
dbext = dbR.fit(pFinal)
dbext.labels_
# plt.hist(dbext.labels_)
# plt.show()

dbdim.labels = dbext.labels_

print(dbdim.labels)

if True:
    dbdim.plot3dclusters(x=dbdim.x[dbdim.FA],
                         y=dbdim.y[dbdim.FA],
                         z=dbdim.z[dbdim.FA],
                         labels=dbdim.labels)

## include orientation in dbscan, not using similarity matrix

In [ ]:
# DBSCAN clustering
dbdim = dbClust(selectSequence='201617')
dbdim.eps=1 
dbdim.min_samples=10

In [ ]:
eps = .25 #* 1e3
min_samples = 5

dbR = sklearn.cluster.DBSCAN(eps = eps, min_samples=min_samples)

In [ ]:
p = np.array([
#               dbdim.x, 
#               dbdim.y, 
#               dbdim.z, 
              dbdim.st1, 
              dbdim.dp1,
              dbdim.st2, 
              dbdim.dp2
#               dbdim.time.astype('float')
             ]).T

p = p[dbdim.FA, :]
for i in np.arange(p.shape[1]):
    view = p[:,i]
    view -= view.mean()
    view *= 1 / np.amax(np.abs(view))
    
pStretch = np.array([1, 1, 1, 
                     1, 1, 1, 1, 1])
for i in np.arange(p.shape[1]):
    p[i] *= pStretch[i]
pTrans = p

In [ ]:
# pTrans = sklearn.preprocessing.StandardScaler().fit_transform(p)
dbext = dbR.fit(pTrans)
dbdim.labels = dbext.labels_
plt.hist(dbdim.labels)
plt.show()

In [ ]:
dbdim.labels = np.zeros(FA.size)-1
dbdim.labels[FA] = dbext.labels_
dbdim.plot3dclusters(pointSize=np.ones(FA.size)*1.5+FA*10)

In [ ]:
if True:
    dbdim.plot3dclusters(x=dbdim.x[dbdim.FA],
                         y=dbdim.y[dbdim.FA],
                         z=dbdim.z[dbdim.FA],
                         labels=dbdim.labels)

There appears to be the problem that s1 of one quake might correspond to s2 of another, yet they will always be compared such that s1 is against s1, not s2. The problem extends to other orientations obviously. Thus, a similarity matrix is probably better.

# Temporal change

In [ ]:
s = 130 * np.pi / 180

vx = np.sin(s)
vy = np.cos(s)

Q = np.array([vx, vy, 0])

In [ ]:
rotateUniVec??

In [ ]:
x = allHypo.x
y = allHypo.y
z = allHypo.z
p = np.array([x, y, z]).T

xPrime = 

# xPrime = np.zeros(x.shape)
# yPrime = np.zeros(y.shape)
# zPrime = np.zeros(z.shape)

# xPrime

# for i in np.arange(x.size):
#     xPrime[i], yPrime[i], zPrime[i] = np.dot()